In [2]:
from dotenv import load_dotenv

load_dotenv()

True

RAGAs 

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embedding_function = OpenAIEmbeddings(model='text-embedding-3-large')
vector_store = Chroma(
    embedding_function=embedding_function,
    collection_name='real_estate_tax',
    persist_directory='./real_estate_tax_collection'
)

retriever = vector_store.as_retriever()

In [5]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm = ChatOpenAI(model='gpt-4o',
                 temperature=0
)


template = """
당신은 질문-응답 작업을 수행하는 어시스턴트입니다.
아래에 제공된 검색된 컨텍스트를 사용하여 질문에 답변하세요.
답을 모를 경우, 모른다고 말하세요.
최대 두 문장으로, 간결하게 답변하세요.

질문: {question}
컨텍스트: {context}
답변:
"""

prompt = ChatPromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

In [6]:
from datasets import Dataset

questions = [
    "종합부동산세의 부과 목적은 무엇이며, 재산세와 어떤 점에서 다른가?", 
    "1세대 1주택자가 종합부동산세 과세대상에서 제외되거나 세액 공제를 받기 위한 요건은 무엇인가?",
    "주택을 3채 이상 보유한 개인에게 적용되는 종합부동산세 세율 구조는 어떻게 되는가?",
]

gt = [
    "종합부동산세는 고액 부동산 보유자에 대한 조세 형평성과 부동산 가격 안정을 목적으로 한다.",
    "1세대 1주택자는 공시가격 요건과 연령 및 보유기간 요건을 충족하면 세액 공제를 받을 수 있다.",
    "3주택 이상 보유한 개인에게는 누진적인 고율의 종합부동산세 세율이 적용된다."
]
answers = []
contexts = []

for query in questions:
    answers.append(rag_chain.invoke(query))
    contexts.append(
        [doc.page_content for doc in retriever.get_relevant_documents(query)]
    )

data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    'ground_truth': gt
}

dataset = Dataset.from_dict(data)


/opt/anaconda3/envs/inflearn_langgraph/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/hq/rb1q4rn512qbbsyy87bd4g9c0000gn/T/ipykernel_7279/488658572.py:20: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  [doc.page_content for doc in retriever.get_relevant_documents(query)]


In [ ]:
from datasets import Dataset

questions = [
    "종합부동산세의 부과 목적은 무엇이며, 재산세와 어떤 점에서 다른가?", 
    "1세대 1주택자가 종합부동산세 과세대상에서 제외되거나 세액 공제를 받기 위한 요건은 무엇인가?",
    "주택을 3채 이상 보유한 개인에게 적용되는 종합부동산세 세율 구조는 어떻게 되는가?",
]

gt = [
    "종합부동산세는 고액 부동산 보유자에 대한 조세 형평성과 부동산 가격 안정을 목적으로 한다.",
    "1세대 1주택자는 공시가격 요건과 연령 및 보유기간 요건을 충족하면 세액 공제를 받을 수 있다.",
    "3주택 이상 보유한 개인에게는 누진적인 고율의 종합부동산세 세율이 적용된다."
]
answers = []
contexts = []

for query in questions:
    answers.append(rag_chain.invoke(query))
    contexts.append(
        [doc.page_content for doc in retriever.get_relevant_documents(query)]
    )

data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    'ground_truth': gt
}

dataset = Dataset.from_dict(data)


/opt/anaconda3/envs/inflearn_langgraph/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/hq/rb1q4rn512qbbsyy87bd4g9c0000gn/T/ipykernel_7279/488658572.py:20: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  [doc.page_content for doc in retriever.get_relevant_documents(query)]


In [31]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
)

result = evaluate(
    dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    ],
     llm=llm  
)

df = result.to_pandas()

/var/folders/hq/rb1q4rn512qbbsyy87bd4g9c0000gn/T/ipykernel_4860/2919819784.py:2: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import (
/var/folders/hq/rb1q4rn512qbbsyy87bd4g9c0000gn/T/ipykernel_4860/2919819784.py:2: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import (
/var/folders/hq/rb1q4rn512qbbsyy87bd4g9c0000gn/T/ipykernel_4860/2919819784.py:2: DeprecationWarning: Importing context_precision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_precision
  from ragas.metrics import (
/var

In [32]:
df

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_precision,context_recall
0,"종합부동산세의 부과 목적은 무엇이며, 재산세와 어떤 점에서 다른가?","[종합부동산세법\n[시행 2023. 4. 18] [법률 제19342호, 2023. ...",종합부동산세의 부과 목적은 고액의 부동산 보유자에게 세금을 부과하여 조세부담의 형평...,종합부동산세는 고액 부동산 보유자에 대한 조세 형평성과 부동산 가격 안정을 목적으로...,0.6,0.848478,1.000000,1.0
1,1세대 1주택자가 종합부동산세 과세대상에서 제외되거나 세액 공제를 받기 위한 요건은...,[과세기준일 현재 만 60세 이상인 1세대 1주택자가 제8조제4항 각 호의 어느 하...,1세대 1주택자가 종합부동산세 과세대상에서 제외되거나 세액 공제를 받기 위해서는 과...,1세대 1주택자는 공시가격 요건과 연령 및 보유기간 요건을 충족하면 세액 공제를 받...,0.2,0.849387,1.000000,1.0
2,주택을 3채 이상 보유한 개인에게 적용되는 종합부동산세 세율 구조는 어떻게 되는가?,[납세의무자가 2주택 이하를 소유한 경우\n\n| 과세표준 ...,주택을 3채 이상 보유한 개인의 종합부동산세 세율 구조는 다음과 같습니다: 과세표준...,3주택 이상 보유한 개인에게는 누진적인 고율의 종합부동산세 세율이 적용된다.,1.0,0.861491,0.833333,1.0


In [ ]:
#!pip install datasets
#!pip install ragas


  Using cached networkx-3.6.1-py3-none-any.whl.metadata (6.8 kB)
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_openai-1.1.4-py3-none-any.whl.metadata (2.6 kB)
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_core-0.3.80-py3-none-any.whl.metadata (3.2 kB)
INFO: pip is still looking at multiple versions of langchain-core to determine which 

G-Eval

In [7]:
from typing import Any


from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

judge_llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

coherence_prompt = ChatPromptTemplate.from_template("""
You are an evaluator.

Evaluate the coherence of the ANSWER.
Focus on logical flow, clarity, and internal consistency.
Do NOT consider factual correctness.

Score from 1 to 5:
1 = incoherent
3 = partially coherent
5 = fully coherent

Return only the score as an integer.

ANSWER:
{answer}
""")

relevance_prompt = ChatPromptTemplate.from_template("""
You are an evaluator.

Evaluate how relevant the ANSWER is to the QUESTION.
Ignore factual correctness.
Focus only on whether the answer addresses the question.

Score from 1 to 5:
1 = completely irrelevant
3 = partially relevant
5 = fully relevant

Return only the score as an integer.

QUESTION:
{question}

ANSWER:
{answer}
""")

groundedness_prompt = ChatPromptTemplate.from_template("""
You are an evaluator.

Evaluate whether the ANSWER is supported by the CONTEXT.
Do not use outside knowledge.
Penalize any information not found in the context.

Score from 1 to 5:
1 = not grounded at all
3 = partially grounded
5 = fully grounded

Return only the score as an integer.

CONTEXT:
{context}

ANSWER:
{answer}
""")


coherence_chain = coherence_prompt | judge_llm | StrOutputParser()
relevance_chain = relevance_prompt | judge_llm | StrOutputParser()
groundedness_chain = groundedness_prompt | judge_llm | StrOutputParser()

coherence_scores = []
relevance_scores = []
groundedness_scores = []

for q, a, ctx in zip(questions, answers, contexts):
    coherence_scores.append(
        int(coherence_chain.invoke({"answer": a}))
    )
    
    relevance_scores.append(
        int(relevance_chain.invoke({
            "question": q,
            "answer": a
        }))
    )
    
    groundedness_scores.append(
        int(groundedness_chain.invoke({
            "context": "\n".join(ctx),
            "answer": a
        }))
    )



In [8]:
import pandas as pd

df_geval = pd.DataFrame({
    "question": questions,
    "answer": answers,
    "geval_coherence": coherence_scores,
    "geval_relevance": relevance_scores,
    "geval_groundedness": groundedness_scores
})

df_geval


,question,answer,geval_coherence,geval_relevance,geval_groundedness
0,"종합부동산세의 부과 목적은 무엇이며, 재산세와 어떤 점에서 다른가?",종합부동산세의 부과 목적은 고액의 부동산 보유자에게 세금을 부과하여 조세부담의 형평...,5,5,5
1,1세대 1주택자가 종합부동산세 과세대상에서 제외되거나 세액 공제를 받기 위한 요건은...,1세대 1주택자가 종합부동산세 과세대상에서 제외되거나 세액 공제를 받기 위해서는 과...,5,5,5
2,주택을 3채 이상 보유한 개인에게 적용되는 종합부동산세 세율 구조는 어떻게 되는가?,주택을 3채 이상 보유한 개인의 종합부동산세 세율 구조는 다음과 같습니다: 과세표준...,5,5,5


In [10]:
contexts

[['종합부동산세법\n[시행 2023. 4. 18] [법률 제19342호, 2023. 4. 18, 일부개정]\n기획재정부 (재산세제과) 044-215-4313\n제1장 총칙\n제1조(목적) 이 법은 고액의 부동산 보유자에 대하여 종합부동산세를 부과하여 부동산보유에 대한 조세부담의 형평성을 제고하고, 부동산의 가격안정을 도모함으로써 지방재정의 균형발전과 국민경제의 건전한 발전에 이바지함을 목적으로 한다.\n제2조(정의) 이 법에서 사용하는 용어의 정의는 다음 각호와 같다. <개정 2005. 12. 31., 2010. 3. 31., 2016. 1. 19., 2018. 12. 31., 2020. 6. 9., 2023. 3. 14.>\n1. "시·군·구"란 함은 「지방자치법」 제2조에 따른 지방자치단체인 시·군 및 자치구(이하 "시·군·구"이라 한다)를 말한다.\n2. "시장·군수·구청장"이라 함은 지방자치단체의 장인 시장·군수 및 자치구의 구청장(이하 "시장·군수·구청장"이라 한다)를 말한다.\n3. "주택"이라 함은 「지방세법」 제104조제3호에 의한 주택을 말한다.\n4. "토지"라 함은 「지방세법」 제104조제1호에 따른 토지를 말한다.\n5. "주택분 재산세"라 함은 「지방세법」 제105조 및 제107조에 따라 주택에 대하여 부과하는 재산세를 말한다.\n6. "토지분 재산세"라 함은 「지방세법」 제105조 및 제107조에 따라 토지에 대하여 부과하는 재산세를 말한다. <삭제 2005. 12. 31.>\n7. "세대"라 함은 주택 또는 토지의 소유자 및 그 배우자와 그들과 생계를 같이하는 가족으로서 대통령령으로 정하는 것을 말한다.\n8. "공시가격"이라 함은 「부동산 가격공시에 관한 법률」에 따라 가격이 공시되는 주택 및 토지에 대하여 같은 법에 따라 공시된 가격을 말한다. 다만, 같은 법에 따라 가격이 공시되지 아니한 경우에는 「지방세법」 제4조제1항 단서 및 같은 조 제2항에 따른 가액으로 한다.\n제3조(과세기준) 종합부동산세의 과세기준일은 「지방세법」

In [12]:
questions

['종합부동산세의 부과 목적은 무엇이며, 재산세와 어떤 점에서 다른가?',
 '1세대 1주택자가 종합부동산세 과세대상에서 제외되거나 세액 공제를 받기 위한 요건은 무엇인가?',
 '주택을 3채 이상 보유한 개인에게 적용되는 종합부동산세 세율 구조는 어떻게 되는가?']

In [14]:
answers

['종합부동산세의 부과 목적은 고액의 부동산 보유자에게 세금을 부과하여 조세부담의 형평성을 제고하고 부동산 가격 안정을 도모하는 것입니다. 재산세와의 차이점은 종합부동산세는 고액의 부동산 보유자에게 추가로 부과되는 세금인 반면, 재산세는 모든 부동산 소유자에게 부과되는 기본적인 세금입니다.',
 '1세대 1주택자가 종합부동산세 과세대상에서 제외되거나 세액 공제를 받기 위해서는 과세기준일 현재 만 60세 이상이거나 해당 주택을 5년 이상 보유해야 하며, 연령과 보유기간에 따라 공제율이 적용됩니다. 만 60세 이상은 연령별로 최대 40%까지, 5년 이상 보유 시 보유기간에 따라 최대 50%까지 공제받을 수 있습니다.',
 '주택을 3채 이상 보유한 개인의 종합부동산세 세율 구조는 다음과 같습니다: 과세표준 3억 원 이하일 경우 1천분의 5, 3억 원 초과 6억 원 이하일 경우 150만 원 + (초과 금액의 1천분의 7), 6억 원 초과 12억 원 이하일 경우 360만 원 + (초과 금액의 1천분의 10), 12억 원 초과 25억 원 이하일 경우 960만 원 + (초과 금액의 1천분의 20), 25억 원 초과 50억 원 이하일 경우 3천560만 원 + (초과 금액의 1천분의 30), 50억 원 초과 94억 원 이하일 경우 1억1천600만 원 + (초과 금액의 1천분의 40), 94억 원 초과일 경우 2억8천660만 원 + (초과 금액의 1천분의 50)입니다.']